In [1]:
#  Cargar datos de entrenamiento
import pandas as pd  #  type: ignore

train_data = pd.read_csv(
    "../files/input/train_data.csv.zip",
    #index_col=False,
    compression="zip",
)


#  Cargar datos de prueba
import pandas as pd  #  type: ignore

test_data = pd.read_csv(
    "../files/input/test_data.csv.zip",
    #index_col=False,
    compression="zip",
)

In [2]:
# Paso 1.
# Realice la limpieza de los datasets:
# - Renombre la columna "default payment next month" a "default".
# - Remueva la columna "ID".
# - Elimine los registros con informacion no disponible.
# - Para la columna EDUCATION, valores > 4 indican niveles superiores
#   de educación, agrupe estos valores en la categoría "others".
#
# Renombre la columna "default payment next month" a "default"
# y remueva la columna "ID".

# - Renombre la columna "default payment next month" a "default".
train_data.rename(columns={'default payment next month': 'default'}, inplace=True)
test_data.rename(columns={'default payment next month': 'default'}, inplace=True)

#  Eliminar la columna ID
train_data.drop(columns=['ID'], inplace=True)
test_data.drop(columns=['ID'], inplace=True)

# Elimine los registros con informacion no disponible.
test_data.dropna(inplace=True)
test_data.isna().sum()

# Para la columna EDUCATION, valores > 4 indican niveles superiores, 
# agrupe estos valores en la categoría "others", es decir igual a 4
# train_data['EDUCATION'] = train_data['EDUCATION'].apply(lambda x: x if x <= 4 else 4)
# test_data['EDUCATION'] = test_data['EDUCATION'].apply(lambda x: x if x <= 4 else 4)


df = test_data.loc[(test_data["EDUCATION"] != 0)]
df = test_data.loc[(test_data["MARRIAGE"] != 0)]
df.loc[test_data["EDUCATION"] > 4, "EDUCATION"] = 4

df = train_data.loc[(train_data["EDUCATION"] != 0)]
df = train_data.loc[(train_data["MARRIAGE"] != 0)]
df.loc[train_data["EDUCATION"] > 4, "EDUCATION"] = 4

C:\Users\danil\AppData\Local\Temp\ipykernel_32112\3232869872.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[test_data["EDUCATION"] > 4, "EDUCATION"] = 4
C:\Users\danil\AppData\Local\Temp\ipykernel_32112\3232869872.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[train_data["EDUCATION"] > 4, "EDUCATION"] = 4


In [3]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.
# Divida los datasets en x_train, y_train, x_test, y_test
# x_train y x_test contienen  todas las columnas excepto la columna default.
# y_train y y_test contiene la variable objetivo default (pago)
x_train = train_data.drop(['default'], axis=1)
y_train = train_data['default']

x_test = test_data.drop(['default'], axis=1)
y_test = test_data['default']

In [4]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las demas variables al intervalo [0, 1].
# - Selecciona las K mejores caracteristicas.
# - Ajusta un modelo de regresion logistica.

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Definir columnas categóricas y numéricas
categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE']
numerical_features = ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
                   'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
                   'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

# Preprocesamiento para las variables categóricas
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

# Preprocesamiento para las variables numéricas
numerical_transformer = MinMaxScaler()

# Combinación de preprocesadores
preprocessor = ColumnTransformer(
    transformers= [
        ("encoder", categorical_transformer, categorical_features),
        ("numerica", numerical_transformer, numerical_features)
    ]
)

# Creación del Pipeline
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("selectkbest", SelectKBest(score_func=f_regression, k=10)),
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

# Ajustar el pipeline con los datos de entrenamiento
pipeline.fit(x_train, y_train)

# Evaluar el modelo en el conjunto de prueba
print("Modelo entrenado. Precisión en test:", pipeline.score(x_test, y_test))

Modelo entrenado. Precisión en test: 0.8205555555555556


In [5]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score
import warnings

warnings.filterwarnings("ignore")
# Definición del grid de hiperparámetros
param_grid = {
    #"feature_selection__k": range(1, 10),  # Diferentes valores de k para probar
    "selectkbest__k": range(1, len(x_train.columns) + 1),  # Diferentes valores de k para probar
    "classifier__C": [0.1, 1, 10],  # 1 poner rango de valores
    "classifier__solver": ["liblinear", "lbfgs"]
    #"classifier__solver": ["liblinear"],  # Solvers disponibles  "saga" para l1 y l2, "lbfgs" para l2, "liblinear" l1
    #"classifier__penalty": ["l1"],  # Penalización l1 o l2
}



# Validación cruzada con 10 splits
grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring="balanced_accuracy", n_jobs=-1, refit=True )
grid_search.fit(x_train, y_train)


# Mejor modelo
print("Mejores parámetros: ", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Evaluación en el conjunto de entrenamiento
y_train_pred = best_model.predict(x_train)  # Predicciones en el conjunto de entrenamiento
balanced_acc_train = balanced_accuracy_score(y_train, y_train_pred)  # Comparar con las etiquetas verdaderas de entrenamiento
print("Balanced Accuracy en el conjunto de entrenamiento:", balanced_acc_train)

# Evaluación en el conjunto de test
y_test_pred = best_model.predict(x_test)  # Predicciones en el conjunto de test
balanced_acc_test = balanced_accuracy_score(y_test, y_test_pred)  # Comparar con las etiquetas verdaderas de test
print("Balanced Accuracy en el conjunto de prueba:", balanced_acc_test)

Mejores parámetros:  {'classifier__C': 1, 'classifier__solver': 'liblinear', 'selectkbest__k': 1}
Balanced Accuracy en el conjunto de entrenamiento: 0.6391959574465568
Balanced Accuracy en el conjunto de prueba: 0.654602898974264


In [6]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

import pickle
import gzip

# Guardar el modelo comprimido con gzip
with gzip.open('../files/models/model.pkl.gz', 'wb') as file:
    pickle.dump(grid_search, file)

In [7]:
from sklearn.metrics import recall_score, balanced_accuracy_score, f1_score, accuracy_score, precision_score

# Predicciones
y_train_pred = best_model.predict(x_train)
y_test_pred = best_model.predict(x_test)


# Métricas
metrics = {
    "Train": {
        "Accuracy": accuracy_score(y_train, y_train_pred),
        "Balanced accuracy": balanced_accuracy_score(y_train, y_train_pred),
        "Precision": precision_score(y_train, y_train_pred),
        "Recall": recall_score(y_train, y_train_pred),
        "F1-Score": f1_score(y_train, y_train_pred)
    },
    "Test":{
        "Accuracy": accuracy_score(y_test, y_test_pred),
        "Balanced accuracy": balanced_accuracy_score(y_test, y_test_pred),
        "Precision": precision_score(y_test, y_test_pred),
        "Recall": recall_score(y_test, y_test_pred),
        "F1-Score": f1_score(y_test, y_test_pred)
    }
}

print(metrics)

{'Train': {'Accuracy': 0.815, 'Balanced accuracy': np.float64(0.6391959574465568), 'Precision': 0.6932966023875115, 'Recall': 0.31944150624074463, 'F1-Score': 0.4373642288196959}, 'Test': {'Accuracy': 0.8303333333333334, 'Balanced accuracy': np.float64(0.654602898974264), 'Precision': 0.7006302521008403, 'Recall': 0.3493975903614458, 'F1-Score': 0.46627053477804964}}


In [8]:
# Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'type': 'metrics', 'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}
#
#
# Paso 7.
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#
from sklearn.metrics import classification_report, balanced_accuracy_score, confusion_matrix
import json

# Función para calcular las métricas de precisión, recall, etc.
def compute_metrics(model, X, y, dataset):
    y_pred = model.predict(X)
    metrics = classification_report(y, y_pred, output_dict=True)
    
    # formato de json
    results = {
        'type': 'metrics',
        'dataset': dataset,
        'precision': metrics['1']['precision'],
        'balanced_accuracy': balanced_accuracy_score(y, y_pred),
        'recall': metrics['1']['recall'],
        'f1_score': metrics['1']['f1-score']
    }
    return results

# Función para calcular la matriz de confusión
def compute_confusion_matrix(model, X, y, dataset):
    cm = confusion_matrix(y, model.predict(X))

    # formato de json
    return {
        'type': 'cm_matrix',
        'dataset': dataset,
        'true_0': {'predicted_0': int(cm[0][0]), 'predicted_1': int(cm[0][1])},
        'true_1': {'predicted_0': int(cm[1][0]), 'predicted_1': int(cm[1][1])}
    }

# Calcula las métricas para entrenamiento y prueba
metrics_list = [
    compute_metrics(grid_search.best_estimator_, x_train, y_train, 'train'),
    compute_metrics(grid_search.best_estimator_, x_test, y_test, 'test')
]

# Calcula la matriz de confusión para entrenamiento y prueba
cm_train = compute_confusion_matrix(grid_search.best_estimator_, x_train, y_train, 'train')
cm_test = compute_confusion_matrix(grid_search.best_estimator_, x_test, y_test, 'test')

# Crear una lista con todas las métricas y matrices
all_results = metrics_list + [cm_train, cm_test]

# Guardar todas las métricas y matrices en el archivo 'metrics.json'
with open('../files/output/metrics.json', 'w') as file:
    for result in all_results:
        file.write(json.dumps(result) + '\n')

#mostrar resultados

print(metrics_list)
print(cm_train)
print(cm_test)
print(all_results)



[{'type': 'metrics', 'dataset': 'train', 'precision': 0.6932966023875115, 'balanced_accuracy': np.float64(0.6391959574465568), 'recall': 0.31944150624074463, 'f1_score': 0.4373642288196959}, {'type': 'metrics', 'dataset': 'test', 'precision': 0.7006302521008403, 'balanced_accuracy': np.float64(0.654602898974264), 'recall': 0.3493975903614458, 'f1_score': 0.46627053477804964}]
{'type': 'cm_matrix', 'dataset': 'train', 'true_0': {'predicted_0': 15605, 'predicted_1': 668}, 'true_1': {'predicted_0': 3217, 'predicted_1': 1510}}
{'type': 'cm_matrix', 'dataset': 'test', 'true_0': {'predicted_0': 6806, 'predicted_1': 285}, 'true_1': {'predicted_0': 1242, 'predicted_1': 667}}
[{'type': 'metrics', 'dataset': 'train', 'precision': 0.6932966023875115, 'balanced_accuracy': np.float64(0.6391959574465568), 'recall': 0.31944150624074463, 'f1_score': 0.4373642288196959}, {'type': 'metrics', 'dataset': 'test', 'precision': 0.7006302521008403, 'balanced_accuracy': np.float64(0.654602898974264), 'recall':